# Object Detection

In [2]:
!git clone https://github.com/ultralytics/yolov5

!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.


Defaulting to user installation because normal site-packages is not writeable


In [3]:
import cv2 as cv
import os
import random
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import xml.etree.ElementTree as ET
from utils import ModelTrainer, displayImage, importImage, plotTrainingHistory

In [4]:
# Get train and validation datasets
images_directory = "../dataset/images"
annotations_directory = "../dataset/annotations"

with open("train.txt") as train:
    train_images_filenames = sorted(train.read().splitlines())

with open("test.txt") as test:
    val_images_filenames = sorted(test.read().splitlines())

# Filter out images that can not be loaded properly
train_images_filenames = [i for i in train_images_filenames if cv.imread(os.path.join(images_directory, i + ".png")) is not None]
val_images_filenames = [i for i in val_images_filenames if cv.imread(os.path.join(images_directory, i + ".png")) is not None]

random.seed(42)
random.shuffle(train_images_filenames)
random.shuffle(val_images_filenames)

print(len(train_images_filenames), len(val_images_filenames))

613 264


## YOLO annotations

In [8]:
import shutil
classes = {
    "trafficlight": 0,
    "stop": 1,
    "speedlimit": 2,
    "crosswalk": 3,
}

os.makedirs("../dataset/labels", exist_ok=True)

os.makedirs("dataset/images/train", exist_ok=True)
os.makedirs("dataset/images/val", exist_ok=True)
os.makedirs("dataset/labels/train", exist_ok=True)
os.makedirs("dataset/labels/val", exist_ok=True)


for img in train_images_filenames:
    shutil.copy(
        f"../dataset/images/{img}.png", f"dataset/images/train/{img}.png")

for img in val_images_filenames:
    shutil.copy(
        f"../dataset/images/{img}.png", f"dataset/images/val/{img}.png")


for path in os.listdir(annotations_directory):
    with open(os.path.join(annotations_directory, path), "r") as xml:
        folder = "train" if path.strip(".xml") in train_images_filenames else "val"
        with open(f"dataset/labels/{folder}/{path.replace('xml', 'txt')}", "w") as txt:
            tree = ET.parse(xml)
            root = tree.getroot()

            size = root.find("size")
            height = int(size.find("height").text)
            width = int(size.find("width").text)

            objects = root.findall("object")

            lines = []
            for object in objects:
                class_index = classes[object.find("name").text]

                xmin = int(object.find("bndbox/xmin").text)
                ymin = int(object.find("bndbox/ymin").text)
                xmax = int(object.find("bndbox/xmax").text)
                ymax = int(object.find("bndbox/ymax").text)

                # middle of bbox
                bbox_x = ((xmax + xmin) / 2) / width
                bbox_y = ((ymax + ymin) / 2) / height
                bbox_width = (xmax - xmin) / width
                bbox_height = (ymax - ymin) / height

                lines.append(
                    f"{class_index} {bbox_x} {bbox_y} {bbox_width} {bbox_height}")

            txt.write("\n".join(lines))


In [13]:
classes = {
    "trafficlight": 0,
    "stop": 1,
    "speedlimit": 2,
    "crosswalk": 3,
}

class TrafficSignDataset(Dataset):
    def __init__(self, annotations_directory, images_filenames, images_directory, transform=None):
        self.annotations_directory = annotations_directory
        self.images_filenames = images_filenames
        self.images_directory = images_directory
        self.transform = transform

    def __len__(self):
        return len(self.images_filenames)

    def __getitem__(self, idx):
        image_filename = self.images_filenames[idx]
        image = cv.imread(os.path.join(
            self.images_directory, image_filename + ".png"))
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        # transform image into 0-1 range
        # note that the ToTensorV2 method from the albumentations library does not automatically convert the image range into 0-1
        image = image / 255.

        if self.transform is not None:
            transformed = self.transform(image=image)
            image = transformed["image"]

        return image.float(), self._get_max_area_class_from_annotion(image_filename)

    def _get_max_area_class_from_annotion(self, filename):
        with open(os.path.join(self.annotations_directory, filename + ".xml")) as xml:
            tree = ET.parse(xml)
            root = tree.getroot()

            max_area_class = "trafficlight"
            max_area = -1

            objects = root.findall("object")
            for object in objects:
                xmin = int(object.find("bndbox/xmin").text)
                ymin = int(object.find("bndbox/ymin").text)
                xmax = int(object.find("bndbox/xmax").text)
                ymax = int(object.find("bndbox/ymax").text)

                area = (xmax - xmin) * (ymax - ymin)

                if area > max_area:
                    max_area = area
                    max_area_class = object.find("name").text

        return classes[max_area_class]

In [6]:
num_classes = len(classes)

# Get CPU or GPU device for training
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Two-Stage

## One-Stage

In [17]:
# Might need to be run on linux/WSL
!python3 yolov5/train.py --batch 10 --epochs 30 --data trafficsigns.yaml
# python yolov5/train.py --batch -1 --epochs 3 --data trafficsigns.yaml --workers 0

SyntaxError: invalid syntax (3741266108.py, line 1)

In [ ]:
# Detection
!python3 yolov5/detect.py --weights yolov5/runs/train/exp22/weights/best.pt --img 640 --conf 0.25 --source dataset/images/train/road2.png

In [8]:
# yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', channels=3, classes=4, autoshape=False)
# yolo_model.to(device)

# yolo = {
#     "model": yolo_model,
#     "name": 'yolov5s',
#     "num_epochs": 10,
#     "loss": nn.CrossEntropyLoss(),  # already includes the Softmax activation
#     "optimizer": torch.optim.SGD(yolo_model.parameters(), lr=1e-3)
# }

# print(yolo_model)

    exitcode = _main(fd, parent_sentinel)
  File "c:\Program Files\Python310\lib\multiprocessing\spawn.py", line 125, in _main
    prepare(preparation_data)
  File "c:\Program Files\Python310\lib\multiprocessing\spawn.py", line 236, in prepare
    _fixup_main_from_path(data['init_main_from_path'])
  File "c:\Program Files\Python310\lib\multiprocessing\spawn.py", line 287, in _fixup_main_from_path
    main_content = runpy.run_path(main_path,
  File "c:\Program Files\Python310\lib\runpy.py", line 269, in run_path
    return _run_module_code(code, init_globals, run_name,
  File "c:\Program Files\Python310\lib\runpy.py", line 96, in _run_module_code
    _run_code(code, mod_globals, init_globals,
  File "c:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\Ricardo\Desktop\VC2022\Part2\yolov5\train.py", line 26, in <module>
    import torch
  File "C:\Users\Ricardo\AppData\Roaming\Python\Python310\site-packages\torch\__init__.py", line 